In [1]:
import torch
from PINN import PINN
from Net import Net
import numpy as np
import pandas as pd
import numpy as np
import time
import os

# set default type double
default_type = torch.float32
torch.set_default_dtype(default_type)

In [2]:
data_dir = f"{os.getcwd()}/pinn_test_data"
# load in data
df_wind_ch4 = pd.read_csv(data_dir + "/wind_ch4.csv")
df_true_emission = pd.read_csv(data_dir + "/selected_controll_release.csv")
source_points = np.load(data_dir + "/source_points.npy") # shape=(n_source, 3)
sensor_points = np.load(data_dir + "/sensor_points.npy") # shape=(n_sensor, 3)
#col_points = np.load(data_dir + "/col_points.npy")  # shape=(n_col, 3)
df_bounds = pd.read_csv(data_dir + "/bounds.csv", dtype='float32')
x_min = df_bounds['x_min'][0]
x_max = df_bounds['x_max'][0]
y_min = df_bounds['y_min'][0]
y_max = df_bounds['y_max'][0]
z_min = df_bounds['z_min'][0]
z_max = df_bounds['z_max'][0]


ws = df_wind_ch4['wind_speed.m/s'].to_numpy() # shape=(N_t,)
wd = df_wind_ch4['wind_direction'].to_numpy() # shape=(N_t,)
ch4 = np.transpose(df_wind_ch4.iloc[:, 3:].to_numpy()) # shape=(N_obs, N_t)
sensor_names = df_wind_ch4.columns[3:]

In [3]:
print(source_points)

[[61.84826691 40.32822479  4.5       ]
 [99.10094831 54.69940709  2.        ]
 [99.89962676 24.72759871  2.        ]
 [23.54499552 57.03946784  2.        ]
 [25.09781584 22.62636785  2.        ]]


In [4]:
model = PINN([20,20,20])
model.set_location(source_points,[1,x_max,y_max,z_max],source_values=[0,10,0,0,0],sigma = .1)
tfinal = 1.

In [5]:
optimizer = torch.optim.Adam(model.net.parameters(), lr=1e-3)
# from torch.optim.lr_scheduler import ExponentialLR

# scheduler = ExponentialLR(optimizer, gamma=0.995)  # Decay LR by 5% every epoch

In [6]:
model.net

Net(
  (hidden): ModuleList(
    (0): Linear(in_features=4, out_features=20, bias=True)
    (1-2): 2 x Linear(in_features=20, out_features=20, bias=True)
    (3): Linear(in_features=20, out_features=1, bias=True)
  )
  (relu): ReLU()
  (tanh): Tanh()
  (leaky_relu): LeakyReLU(negative_slope=0.1)
)

In [7]:
n= int(1e2)
sn = 3000
best_loss = np.inf
print_freq = 1
max_epochs = int(1e3)
sample_freq = 5
n_source_samples = int(2e3)

standard_dev = 1e-3

for epoch in range(max_epochs):
    start_time = time.time()
    if epoch % sample_freq == 0:
        source_collocation_points = model.source_points(n_source_samples,standard_dev)
        ic_col = torch.cat([torch.rand(sn,1)*tfinal*.1, torch.rand(sn,1)*x_max, torch.rand(sn,1)*y_max, torch.rand(sn,1)*z_max], dim=1)

        collocation_points = torch.cat([torch.rand(n,1)*tfinal, torch.rand(n,1)*x_max, torch.rand(n,1)*y_max, torch.rand(n,1)*z_max], dim=1)
        collocation_points = torch.empty(0,4)
        # ic_col = torch.empty(0,4)
        ic_col.requires_grad_()
        collocation_points = torch.cat([collocation_points,ic_col,source_collocation_points])
        wind_vector = torch.ones(len(collocation_points),2)*1.0
        wind_vector[:,0] = -1
    optimizer.zero_grad()

    loss_1 ,pde_1 = model.loss_function(collocation_points,wind_vector) # PDE residual loss
    # loss_2,pde_2 = model.loss_function(source_collocation_points, uv_points, source_term = source_values) # source term PDE residual loss 
    # loss_3 ,pde_3 = model.loss_function(torch.concat([collocation_points,source_collocation_points]),torch.concat([collocation_points,source_collocation_points]))

    loss = loss_1 
    
    loss.backward()

    # compute norm of gradient of the network
    grad_norm = 0
    for p in model.net.parameters():
        # if p.grad.data is not None:
        if p.grad is not None:
            grad_norm += p.grad.data.norm().item()**2
    grad_norm = grad_norm**0.5


    if loss.item() < best_loss:
        torch.save(model,'best_mod.m')
    optimizer.step()

    end_time = time.time()
    epoch_time = end_time - start_time
    # scheduler.step()

    if epoch % print_freq == 0:

        # print epoch and loss using %1.3e format
        print('epoch: %d, loss: %1.3e, grad_norm: %1.3e, pde_res: %1.3e, time: %1.3e' % (epoch, loss.item(), grad_norm, loss_1.item(), epoch_time))



epoch: 0, loss: 1.347e+02, grad_norm: 1.075e+03, pde_res: 1.347e+02, time: 1.422e-01
epoch: 1, loss: 1.284e+02, grad_norm: 9.455e+02, pde_res: 1.284e+02, time: 8.654e-02
epoch: 2, loss: 1.230e+02, grad_norm: 8.178e+02, pde_res: 1.230e+02, time: 7.780e-02
epoch: 3, loss: 1.183e+02, grad_norm: 6.911e+02, pde_res: 1.183e+02, time: 8.981e-02
epoch: 4, loss: 1.144e+02, grad_norm: 5.650e+02, pde_res: 1.144e+02, time: 7.805e-02
epoch: 5, loss: 1.114e+02, grad_norm: 4.354e+02, pde_res: 1.114e+02, time: 7.463e-02
epoch: 6, loss: 1.091e+02, grad_norm: 3.199e+02, pde_res: 1.091e+02, time: 7.945e-02
epoch: 7, loss: 1.076e+02, grad_norm: 2.123e+02, pde_res: 1.076e+02, time: 7.467e-02
epoch: 8, loss: 1.066e+02, grad_norm: 1.145e+02, pde_res: 1.066e+02, time: 7.665e-02
epoch: 9, loss: 1.061e+02, grad_norm: 4.446e+01, pde_res: 1.061e+02, time: 7.346e-02
epoch: 10, loss: 1.057e+02, grad_norm: 8.026e+01, pde_res: 1.057e+02, time: 7.407e-02
epoch: 11, loss: 1.059e+02, grad_norm: 1.495e+02, pde_res: 1.059

In [8]:
source_collocation_points

tensor([[5.1330e-02, 9.9101e+01, 5.4699e+01, 2.0006e+00],
        [4.4884e-01, 9.9102e+01, 5.4698e+01, 1.9981e+00],
        [8.0663e-01, 9.9100e+01, 5.4700e+01, 2.0001e+00],
        ...,
        [9.5942e-03, 9.9101e+01, 5.4702e+01, 2.0011e+00],
        [3.4279e-01, 9.9100e+01, 5.4699e+01, 1.9991e+00],
        [4.2094e-01, 9.9102e+01, 5.4700e+01, 1.9988e+00]])

In [9]:
model.net

Net(
  (hidden): ModuleList(
    (0): Linear(in_features=4, out_features=20, bias=True)
    (1-2): 2 x Linear(in_features=20, out_features=20, bias=True)
    (3): Linear(in_features=20, out_features=1, bias=True)
  )
  (relu): ReLU()
  (tanh): Tanh()
  (leaky_relu): LeakyReLU(negative_slope=0.1)
)

In [10]:
model = torch.load('best_mod.m', map_location=torch.device('cpu'))

/var/folders/8t/rfcrq95s3cj0_l1g0lq3_v540000gn/T/ipykernel_16126/3839396481.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load('best_mod.m', map_location

In [11]:
import matplotlib.pyplot as plt

# load the best model 
Z_value = 2
# net.load_state_dict(torch.load('best_model.pth'))

# Define the grid and time steps
n= 500
x_grid = np.linspace(-x_max, 2.0*x_max, n)
y_grid = np.linspace(-y_max, 2.0*y_max, n)
z_grid = np.linspace(0, 2, n)

X, Y, = np.meshgrid(x_grid, y_grid)
Z= X * 0 + Z_value

grid_points = np.vstack([X.ravel(), Y.ravel(), Z.ravel()]).T
# grid_points = np.vstack([X.ravel(), Y.ravel(), Z.ravel()]).T  # Flatten the grid
grid_points = torch.tensor(grid_points, dtype=torch.float32)

time_steps = np.linspace(0, tfinal, 50)  # 10 time steps from 0 to 1


'''
# Plot the concentration over time
for t in time_steps:
    t_tensor = torch.full((grid_points.shape[0], 1), t, dtype=torch.float32)  # Time input
    concentration = net(grid_points, t_tensor).cpu().detach().numpy().reshape(100, 100)  # Predict and reshape
    
    plt.figure()
    plt.contourf(X, Y, concentration, levels=50, cmap='viridis', vmin=0, vmax=10)  # Plot concentration as a contour plot
    plt.plot(source_loc[0,0].cpu(), source_loc[0,1].cpu(), 'ro', label='Source Location')  # Plot the source location
    plt.colorbar(label='Concentration')
    # fix colorbar from 0 to 1 
    # plt.clim(0, 10.0)  # Set colorbar limits
    plt.title(f"Gas Concentration at t = {t:.2f}")
    plt.xlabel('x')
    plt.ylabel('y')
    # fix colorbar 
    # plt.savefig(f"concentration_t_{t:.2f}.png")  # Save the plot as an image
'''

'\n# Plot the concentration over time\nfor t in time_steps:\n    t_tensor = torch.full((grid_points.shape[0], 1), t, dtype=torch.float32)  # Time input\n    concentration = net(grid_points, t_tensor).cpu().detach().numpy().reshape(100, 100)  # Predict and reshape\n    \n    plt.figure()\n    plt.contourf(X, Y, concentration, levels=50, cmap=\'viridis\', vmin=0, vmax=10)  # Plot concentration as a contour plot\n    plt.plot(source_loc[0,0].cpu(), source_loc[0,1].cpu(), \'ro\', label=\'Source Location\')  # Plot the source location\n    plt.colorbar(label=\'Concentration\')\n    # fix colorbar from 0 to 1 \n    # plt.clim(0, 10.0)  # Set colorbar limits\n    plt.title(f"Gas Concentration at t = {t:.2f}")\n    plt.xlabel(\'x\')\n    plt.ylabel(\'y\')\n    # fix colorbar \n    # plt.savefig(f"concentration_t_{t:.2f}.png")  # Save the plot as an image\n'

In [ ]:
# save as a GIF
import imageio
import os

images = []
# source_loc = torch.tensor([[.5,.5,.5]])
for t in time_steps:
    t_tensor = torch.full((grid_points.shape[0], 1), t, dtype=default_type)  # Time input
    concentration = model.forward(torch.cat([t_tensor,grid_points],dim=1),scaled=False).cpu().detach().numpy().reshape(n, n)  # Predict and reshape
    
    plt.figure()
    plt.contourf(X, Y, concentration, levels=50, cmap='viridis', vmin=0, vmax=50)  # Plot concentration as a contour plot
    plt.plot(model.source_locs[1,0], model.source_locs[1,1], 'ro', label='Source Location')  # Plot the source location
    plt.colorbar(label='Concentration')
    # plt.title(f"Gas Concentration at t = {t:.2f}")
    plt.title(f"t = {t:.2f}")

    plt.xlabel('x')
    plt.ylabel('y')
    plt.savefig(f"concentration_t_{t:.2f}.png")  # Save the plot as an image
    plt.close()
    
    images.append(imageio.imread(f"concentration_t_{t:.2f}.png"))  # Append the image to the list
    os.remove(f"concentration_t_{t:.2f}.png")  # Remove the image file

imageio.mimsave(f'test_visual.gif', images)  # Save the images as a GIF

/var/folders/8t/rfcrq95s3cj0_l1g0lq3_v540000gn/T/ipykernel_16126/322651465.py:23: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  images.append(imageio.imread(f"concentration_t_{t:.2f}.png"))  # Append the image to the list


In [ ]:
model.source_mixture
source_term = torch.tensor(np.exp(model.source_mixture.score_samples(model.source_locs)))
source_term

In [ ]:
print(model.source_mixture.weights_)

In [ ]:
model.net